# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.127348e+02     1.575583e+00
 * time: 0.10928797721862793
     1     1.177790e+01     9.299616e-01
 * time: 0.8744189739227295
     2    -1.184639e+01     1.093420e+00
 * time: 1.0244719982147217
     3    -3.411810e+01     8.175835e-01
 * time: 1.1793339252471924
     4    -4.806876e+01     5.426639e-01
 * time: 1.3300020694732666
     5    -5.707555e+01     1.935239e-01
 * time: 1.4779210090637207
     6    -5.982223e+01     1.278914e-01
 * time: 1.5927879810333252
     7    -6.084340e+01     6.086012e-02
 * time: 1.7011420726776123
     8    -6.125599e+01     8.501604e-02
 * time: 1.8167569637298584
     9    -6.156780e+01     3.165893e-02
 * time: 1.9327919483184814
    10    -6.176896e+01     2.637364e-02
 * time: 2.042085886001587
    11    -6.192370e+01     2.238485e-02
 * time: 2.1573050022125244
    12    -6.199340e+01     2.043105e-02
 * time: 2.2719030380249023
    13    -6.206863e+01     1.712858e-02
 * time: 2.3860609531

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671074
    AtomicLocal         -18.8557698
    AtomicNonlocal      14.8522653
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485384 
    Xc                  -19.3336825

    total               -62.261666460876
